In [ ]:
!pip install qiskit qiskit-aer matplotlib numpy

In [ ]:
import numpy as np
from qiskit import QuantumCircuit, QuantumRegister
from qiskit.quantum_info import Statevector

class SimpleGroverMarker:
    """
    Simple Grover marker oracle for n=2
    f(x) = 2*x0 + x1, t = 1, C(x) = x0 - x1
    Flips output when f(x) > 1 AND C(x) >= 0
    """

    def __init__(self):
        self.n = 2

    def evaluate_f(self, x):
        """f(x) = 2*x0 + x1"""
        return 2*x[0] + x[1]

    def evaluate_C(self, x):
        """C(x) = x0 - x1"""
        return x[0] - x[1]

    def should_flip(self, x):
        """Check if output should be flipped"""
        return (self.evaluate_f(x) > 1) and (self.evaluate_C(x) >= 0)

    def construct_oracle(self):
        """Construct oracle that flips output only for states [1,0] and [1,1]"""
        # Quantum registers: 2 input qubits, 1 output qubit
        qr = QuantumRegister(3, 'q')
        qc = QuantumCircuit(qr)

        # For n=2, we only have 4 possible states:
        # [0,0]: f=0, C=0 → No flip
        # [0,1]: f=1, C=-1 → No flip
        # [1,0]: f=2, C=1 → Flip ✓
        # [1,1]: f=3, C=0 → Flip ✓

        # Flip output for state |10⟩ (x0=1, x1=0)
        # Control: x0=1, x1=0
        qc.x(1)  # Flip x1 to make control work
        qc.mcx([qr[0], qr[1]], qr[2])  # This now controls on x0=1 AND x1=1 (after x gate)
        qc.x(1)  # Uncompute x1

        # Flip output for state |11⟩ (x0=1, x1=1)
        qc.mcx([qr[0], qr[1]], qr[2])  # Control on x0=1 AND x1=1

        return qc

def test_simple_oracle():
    """Test the simple oracle"""
    print("=== Quantum-Grover-CPBO-Oracle-Test N=2 ===\n")

    oracle = SimpleGroverMarker()
    circuit = oracle.construct_oracle()

    print("Circuit diagram:")
    print(circuit.draw())

    print(f"\nNumber of qubits: {circuit.num_qubits}")
    print(f"Number of gates: {circuit.size()}")
    print(f"Circuit depth: {circuit.depth()}")

    print("\n=== Testing all inputs ===")

    # Test all 4 possible input states
    test_cases = [
        ([0, 0], "00: f=0, C=0 → No flip"),
        ([0, 1], "01: f=1, C=-1 → No flip"),
        ([1, 0], "10: f=2, C=1 → Flip ✓"),
        ([1, 1], "11: f=3, C=0 → Flip ✓")
    ]

    all_correct = True

    for i, (input_bits, description) in enumerate(test_cases):
        # Create test circuit
        test_qc = QuantumCircuit(3)

        # Set input state
        if input_bits[0] == 1:
            test_qc.x(0)
        if input_bits[1] == 1:
            test_qc.x(1)

        # Add oracle
        test_qc.compose(circuit, inplace=True)

        # Simulate
        statevector = Statevector.from_instruction(test_qc)

        # Check output qubit (qubit 2)
        output_flipped = False
        for idx, amplitude in enumerate(statevector):
            if abs(amplitude) > 1e-10:
                state_bits = format(idx, '03b')
                # If input matches and output qubit is 1
                if state_bits[0] == str(input_bits[0]) and state_bits[1] == str(input_bits[1]) and state_bits[2] == '1':
                    output_flipped = True
                    break

        expected = oracle.should_flip(input_bits)
        match = output_flipped == expected

        print(f"Input {input_bits}: {description}")
        print(f"  Output flipped: {output_flipped}, Expected: {expected}, Match: {match}\n")

        if not match:
            all_correct = False


    return circuit, all_correct

# Run the test
circuit, success = test_simple_oracle()

if success:
    print("Oracle  for n=2")
    print(" All test cases verified with quantum simulation")